In [1]:
#1 read file and display head 3 trade rows
import pandas as pd
df_positions = pd.read_csv("/Users/kai/Downloads/TradeHistory-IG-(01-11-2018)-(22-11-2025).csv")
print(df_positions[df_positions['Activity']=="TRADE"].head(3))

     TextDate      Time Activity                            Market Direction  \
0  19-11-2025  19:46:00    TRADE         Okta Inc (Extended Hours)       BUY   
3  23-10-2025  19:52:15    TRADE  MP Materials Corp (All Sessions)       BUY   
6  01-10-2025  15:21:33    TRADE            Intuitive Surgical Inc       BUY   

   Quantity    Price Currency  Consideration  Commission  Charges  \
0      10.0   8018.0      USD         -801.8         0.0      0.0   
3      25.0   6880.0      USD        -1720.0         0.0      0.0   
6       5.0  43800.0      USD        -2190.0         0.0      0.0   

   Cost/Proceeds  Conversion rate Order type Venue ID Settled?  \
0        -618.76         0.771715      LIMIT     XLON        Y   
3       -1300.05         0.755841      LIMIT     XOFF        Y   
6       -1633.80         0.746027      LIMIT     XOFF        Y   

  Settlement date             Order ID  
0      20-11-2025   07CLE:2064083~8779  
3      24-10-2025  07CLE:2051521~10315  
6      02-10-2

In [2]:
#2 calculate positions and their pnl
## read json to dictionary 
import json
with open('/Users/kai/github/st-my-investment/company_name_to_ticker.json', 'r') as f:
    symbols = json.load(f)
## match symbols with df_positions['Market'] to get ticker
df_positions['Ticker'] = df_positions['Market'].map(symbols)
## find Market values that are not in symbols
missing_symbols = df_positions[~df_positions['Market'].isin(symbols.keys())]['Market'].unique()
print("Missing symbols:", missing_symbols)

Missing symbols: []


In [3]:
import requests
# from requests.adapters import HTTPAdapter
# from urllib3.util.retry import Retry

url = "https://www.sec.gov/files/company_tickers.json"
session = requests.Session()
session.headers.update({
    "User-Agent": "MyApp/1.0 (contact: you@example.com) Mozilla/5.0 (Macintosh)",
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Referer": "https://www.sec.gov/",
    "Accept-Language": "en-US,en;q=0.9",
})

# retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[429,500,502,503,504])
# session.mount("https://", HTTPAdapter(max_retries=retries))

resp = session.get(url, timeout=10, allow_redirects=True)
print(resp.status_code)
resp.raise_for_status()
data = resp.json()

200


In [4]:
data

{'0': {'cik_str': 1045810, 'ticker': 'NVDA', 'title': 'NVIDIA CORP'},
 '1': {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'},
 '2': {'cik_str': 1652044, 'ticker': 'GOOGL', 'title': 'Alphabet Inc.'},
 '3': {'cik_str': 789019, 'ticker': 'MSFT', 'title': 'MICROSOFT CORP'},
 '4': {'cik_str': 1018724, 'ticker': 'AMZN', 'title': 'AMAZON COM INC'},
 '5': {'cik_str': 1730168, 'ticker': 'AVGO', 'title': 'Broadcom Inc.'},
 '6': {'cik_str': 1326801, 'ticker': 'META', 'title': 'Meta Platforms, Inc.'},
 '7': {'cik_str': 1318605, 'ticker': 'TSLA', 'title': 'Tesla, Inc.'},
 '8': {'cik_str': 1067983,
  'ticker': 'BRK-B',
  'title': 'BERKSHIRE HATHAWAY INC'},
 '9': {'cik_str': 104169, 'ticker': 'WMT', 'title': 'Walmart Inc.'},
 '10': {'cik_str': 59478, 'ticker': 'LLY', 'title': 'ELI LILLY & Co'},
 '11': {'cik_str': 19617, 'ticker': 'JPM', 'title': 'JPMORGAN CHASE & CO'},
 '12': {'cik_str': 1403161, 'ticker': 'V', 'title': 'VISA INC.'},
 '13': {'cik_str': 884394, 'ticker': 'SPY', 'title': 'S

In [5]:
sec_symbols = {item['ticker']: item['title'] for item in data.values()}
def find_potential_matches(missing_symbols, sec_symbols):
    potential_matches = {} 
    for i in missing_symbols:
        first_word_from_missing_symbols = i.split()[0]
        # search all sec_symbols values that contain i
        matches = {k: v for k, v in sec_symbols.items() if first_word_from_missing_symbols.lower() in v.lower()}
        potential_matches[i] = next(iter(matches), None) # first (key, value) pair
        # print(f"Matches for '{i}': {potential_matches[i]}")
    return potential_matches
new_tickers = find_potential_matches(missing_symbols, sec_symbols)
new_tickers

{}

In [6]:
# Update df_positions['Ticker'] with the potential matches
for market, ticker in new_tickers.items():
    df_positions.loc[df_positions['Market'] == market, 'Ticker'] = ticker
df_positions

,TextDate,Time,Activity,Market,Direction,Quantity,Price,Currency,Consideration,Commission,Charges,Cost/Proceeds,Conversion rate,Order type,Venue ID,Settled?,Settlement date,Order ID,Ticker
0,19-11-2025,19:46:00,TRADE,Okta Inc (Extended Hours),BUY,10.00,8018.00,USD,-801.80,0.0,0.0,-618.76,0.771715,LIMIT,XLON,Y,20-11-2025,07CLE:2064083~8779,OKTA
1,10-11-2025,09:06:27,CORPORATE ACTION,PayPoint PLC,SELL,-0.84,645.43,GBP,5.42,NaN,0.0,5.42,1.000000,LIMIT,NaN,Y,11-11-2025,Z8750-BL1538131,PAY.L
2,10-11-2025,09:06:24,CORPORATE ACTION,PayPoint PLC,BUY,0.84,0.00,GBP,0.00,NaN,0.0,0.00,1.000000,LIMIT,NaN,Y,11-11-2025,Z8750-BL1537968,PAY.L
3,23-10-2025,19:52:15,TRADE,MP Materials Corp (All Sessions),BUY,25.00,6880.00,USD,-1720.00,0.0,0.0,-1300.05,0.755841,LIMIT,XOFF,Y,24-10-2025,07CLE:2051521~10315,MP
4,20-10-2025,07:24:09,CORPORATE ACTION,PayPoint PLC,BUY,553.00,0.00,GBP,0.00,NaN,0.0,0.00,1.000000,LIMIT,NaN,Y,20-10-2025,Z8750-BL1530857,PAY.L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,26-06-2019,14:56:20,TRADE,MSCI Inc,BUY,10.00,23000.00,USD,-2300.00,-15.0,0.0,-1834.74,0.792543,LIMIT,XOFF,Y,28-06-2019,07CLE:788407~9402,MSCI
302,20-05-2019,15:26:36,TRADE,Xilinx Inc,BUY,20.00,9900.00,USD,-1980.00,-15.0,0.0,-1574.26,0.789099,LIMIT,XOFF,Y,22-05-2019,07CLE:770949~9695,XLNX
303,20-05-2019,14:37:06,TRADE,iShares MSCI China A ETF,BUY,667.00,379.00,USD,-2527.93,-8.0,0.0,-2004.39,0.789734,LIMIT,XOFF,Y,22-05-2019,07CLE:770953~7927,IASH.L
304,01-05-2019,17:37:20,TRADE,Zscaler Inc,BUY,60.00,6735.00,USD,-4041.00,-15.0,0.0,-3116.59,0.768390,LIMIT,XOFF,Y,03-05-2019,07CLE:762643~8885,ZS


In [7]:
from datetime import datetime
import pandas as pd

# convert 'TextDate' to datetime
df_positions['Date'] = pd.to_datetime(df_positions['TextDate'], errors='coerce', dayfirst=True)
# Get current quarter start date
today = datetime.now()
current_quarter = pd.Period(today, freq='Q')
quarter_start = current_quarter.start_time

# Use it in your filter
df_trades_this_quarter = df_positions[(df_positions['Activity']=="TRADE") & (df_positions['Date'] >= quarter_start)]
df_trades_this_quarter

,TextDate,Time,Activity,Market,Direction,Quantity,Price,Currency,Consideration,Commission,Charges,Cost/Proceeds,Conversion rate,Order type,Venue ID,Settled?,Settlement date,Order ID,Ticker,Date
0,19-11-2025,19:46:00,TRADE,Okta Inc (Extended Hours),BUY,10.0,8018.0,USD,-801.8,0.0,0.0,-618.76,0.771715,LIMIT,XLON,Y,20-11-2025,07CLE:2064083~8779,OKTA,2025-11-19
3,23-10-2025,19:52:15,TRADE,MP Materials Corp (All Sessions),BUY,25.0,6880.0,USD,-1720.0,0.0,0.0,-1300.05,0.755841,LIMIT,XOFF,Y,24-10-2025,07CLE:2051521~10315,MP,2025-10-23
6,01-10-2025,15:21:33,TRADE,Intuitive Surgical Inc,BUY,5.0,43800.0,USD,-2190.0,0.0,0.0,-1633.80,0.746027,LIMIT,XOFF,Y,02-10-2025,07CLE:2043174~8187,ISRG,2025-10-01
7,01-10-2025,15:00:43,TRADE,Atlassian Corp PLC,BUY,5.0,15800.0,USD,-790.0,0.0,0.0,-588.70,0.745185,LIMIT,XOFF,Y,02-10-2025,07CLE:2043174~7578,TEAM,2025-10-01


In [8]:
df_current_positions = df_positions.groupby('Ticker').agg({'Quantity':'sum', 'Market': 'last', 'Cost/Proceeds': 'sum', 'Charges': 'sum', 'Commission': 'sum', 'Currency': 'last'})
df_current_positions = df_current_positions[df_current_positions['Quantity'] != 0]
df_current_positions['Costs'] = df_current_positions['Cost/Proceeds'] + df_current_positions['Charges'] + df_current_positions['Commission']
df_current_positions

,Quantity,Market,Cost/Proceeds,Charges,Commission,Currency,Costs
Ticker,,,,,,,
AAPL,15.0,Apple Inc (All Sessions),-2170.10,0.00,0.0,USD,-2170.10
AFX.DE,10.0,Carl Zeiss Meditec AG,-437.79,0.00,0.0,EUR,-437.79
ATKR,40.0,Atkore International Group Inc,-2411.73,0.00,0.0,USD,-2411.73
BL,70.0,BlackLine Inc,-4500.08,0.00,0.0,USD,-4500.08
BX,20.0,Blackstone Group LP,-1215.01,-0.06,0.0,USD,-1215.07
CTRA,85.0,Coterra Enercoterra energy Inc,-1886.41,0.00,0.0,USD,-1886.41
FDS,10.0,FactSet Research Systems Inc,-2161.79,-0.03,0.0,USD,-2161.82
GILD,10.0,Gilead Sciences Inc (All Sessions),567.71,-0.14,0.0,USD,567.57
GOLD,120.0,Barrick Gold Corp (US),-1856.21,-0.01,-10.0,USD,-1866.22


In [9]:
## get total investment value

### get current price for each ticker
from getEODprice import getEODpriceUK, getEODpriceUSA 
def get_current_price(tickers: list) -> dict:
    eu_tickers = [ticker for ticker in tickers if ticker.endswith('.L') or ticker.endswith('.DE')]
    us_tickers = [ticker for ticker in tickers if '.' not in ticker]
    us_tickers_positions = getEODpriceUSA(us_tickers)
    eu_tickers_positions = getEODpriceUK(eu_tickers)
    return {**us_tickers_positions, **eu_tickers_positions}
# get_current_price(df_current_positions.index.tolist())
current_prices = get_current_price(df_current_positions.index.tolist())
current_prices

2025-12-16
2025-12-16
2025-12-16


{'MP': '53.42000',
 'SNPS': '463.34000',
 'GOLD': '32.16000',
 'KALU': '111.79000',
 'SKLZ': '5.19000',
 'TMDX': '125.040001',
 'GILD': '118.78000',
 'MS': '176.50999',
 'AAPL': '274.60999',
 'FDS': '293',
 'KOPN': '2.5',
 'ISRG': '554.58002',
 'BX': '152.089996',
 'BL': '57.53000',
 'LITE': '316.14999',
 'QDEL': '28.88000',
 'SOLV': '80.59000',
 'ATKR': '64.050003',
 'TROX': '4.41000',
 'OKTA': '90.59000',
 'MMM': '163.20000',
 'VEEV': '219.30000',
 'NVDA': '177.72000',
 'MOS': '23.46000',
 'MSCI': '551.22998',
 'TEAM': '159.56000',
 'U': '44.57000',
 'LUV': '42.17000',
 'CTRA': '25.30000',
 'LRCX': '163.25999',
 'IVZ': '26.030001',
 'AFX.DE': np.float64(0.39939998626708983),
 'PAY.L': np.float64(4.69),
 'SDR.L': np.float64(3.9120001220703124)}

In [10]:
# add current price to df_current_positions

df_current_positions['Current Price'] = df_current_positions.index.map(current_prices)
df_current_positions['Quantity'] = pd.to_numeric(df_current_positions['Quantity'], errors='coerce')
df_current_positions['Current Price'] = pd.to_numeric(df_current_positions['Current Price'], errors='coerce')

df_current_positions['Market Value'] = df_current_positions['Quantity'] * df_current_positions['Current Price']
df_current_positions

,Quantity,Market,Cost/Proceeds,Charges,Commission,Currency,Costs,Current Price,Market Value
Ticker,,,,,,,,,
AAPL,15.0,Apple Inc (All Sessions),-2170.10,0.00,0.0,USD,-2170.10,274.609990,4119.14985
AFX.DE,10.0,Carl Zeiss Meditec AG,-437.79,0.00,0.0,EUR,-437.79,0.399400,3.99400
ATKR,40.0,Atkore International Group Inc,-2411.73,0.00,0.0,USD,-2411.73,64.050003,2562.00012
BL,70.0,BlackLine Inc,-4500.08,0.00,0.0,USD,-4500.08,57.530000,4027.10000
BX,20.0,Blackstone Group LP,-1215.01,-0.06,0.0,USD,-1215.07,152.089996,3041.79992
CTRA,85.0,Coterra Enercoterra energy Inc,-1886.41,0.00,0.0,USD,-1886.41,25.300000,2150.50000
FDS,10.0,FactSet Research Systems Inc,-2161.79,-0.03,0.0,USD,-2161.82,293.000000,2930.00000
GILD,10.0,Gilead Sciences Inc (All Sessions),567.71,-0.14,0.0,USD,567.57,118.780000,1187.80000
GOLD,120.0,Barrick Gold Corp (US),-1856.21,-0.01,-10.0,USD,-1866.22,32.160000,3859.20000


In [11]:
# get exchange rates for GBP and EUR to USD
GBP = getEODpriceUK(['GBPUSD=X', 'GBPEUR=X'])
GBP

2025-12-16
2025-12-16


{'GBPUSD=X': np.float64(0.013328713178634644),
 'GBPEUR=X': np.float64(0.011375999450683594)}

In [12]:
# calculate total market value in GBP, if ticker with no . is USD, if has . but not .L is EUR
def convert_to_gbp(row):
    if row.name.endswith('.L'):
        return row['Market Value']  # already in GBP
    elif row.name.endswith('.DE'):
        eur_to_gbp = GBP['GBPEUR=X']
        return row['Market Value'] / eur_to_gbp / 100 # convert EUR to GBP
    else:
        usd_to_gbp = GBP['GBPUSD=X']
        return row['Market Value'] / usd_to_gbp / 100 # convert USD to GBP
df_current_positions['Market Value GBP'] = df_current_positions.apply(convert_to_gbp, axis=1)
df_current_positions

,Quantity,Market,Cost/Proceeds,Charges,Commission,Currency,Costs,Current Price,Market Value,Market Value GBP
Ticker,,,,,,,,,,
AAPL,15.0,Apple Inc (All Sessions),-2170.10,0.00,0.0,USD,-2170.10,274.609990,4119.14985,3090.433258
AFX.DE,10.0,Carl Zeiss Meditec AG,-437.79,0.00,0.0,EUR,-437.79,0.399400,3.99400,3.510900
ATKR,40.0,Atkore International Group Inc,-2411.73,0.00,0.0,USD,-2411.73,64.050003,2562.00012,1922.166143
BL,70.0,BlackLine Inc,-4500.08,0.00,0.0,USD,-4500.08,57.530000,4027.10000,3021.371940
BX,20.0,Blackstone Group LP,-1215.01,-0.06,0.0,USD,-1215.07,152.089996,3041.79992,2282.140728
CTRA,85.0,Coterra Enercoterra energy Inc,-1886.41,0.00,0.0,USD,-1886.41,25.300000,2150.50000,1613.434074
FDS,10.0,FactSet Research Systems Inc,-2161.79,-0.03,0.0,USD,-2161.82,293.000000,2930.00000,2198.261723
GILD,10.0,Gilead Sciences Inc (All Sessions),567.71,-0.14,0.0,USD,567.57,118.780000,1187.80000,891.158797
GOLD,120.0,Barrick Gold Corp (US),-1856.21,-0.01,-10.0,USD,-1866.22,32.160000,3859.20000,2895.403291


In [13]:
Total_market_value_gbp = df_current_positions['Market Value GBP'].sum()
USD_market_value_in_gbp = df_current_positions[df_current_positions['Currency']=='USD']['Market Value GBP'].sum()
EUR_market_value_in_gbp = df_current_positions[df_current_positions['Currency']=='EUR']['Market Value GBP'].sum()
GBP_market_value_in_gbp = df_current_positions[df_current_positions['Currency']=='GBP']['Market Value GBP'].sum()
Total_market_value_gbp, USD_market_value_in_gbp, EUR_market_value_in_gbp, GBP_market_value_in_gbp

(np.float64(193489.66487462452),
 np.float64(187371.78386457177),
 np.float64(3.510900189460624),
 np.float64(6114.370109863281))

In [14]:
from plotly import express as px
# plot a pie chart of market value by currency USD_market_value_in_gbp, EUR_market_value_in_gbp, GBP_market_value_in_gbp
fig = px.pie(names=['USD', 'EUR', 'GBP'], values=[USD_market_value_in_gbp, EUR_market_value_in_gbp, GBP_market_value_in_gbp], title='Market Value by Currency in GBP')
fig.show()

In [15]:
df_current_positions['PandL GBP'] = df_current_positions['Market Value GBP'] + df_current_positions['Costs']
df_present_positions = df_current_positions[['Market', 'Quantity', 'Market Value GBP', 'PandL GBP']]
df_present_positions

,Market,Quantity,Market Value GBP,PandL GBP
Ticker,,,,
AAPL,Apple Inc (All Sessions),15.0,3090.433258,920.333258
AFX.DE,Carl Zeiss Meditec AG,10.0,3.510900,-434.279100
ATKR,Atkore International Group Inc,40.0,1922.166143,-489.563857
BL,BlackLine Inc,70.0,3021.371940,-1478.708060
BX,Blackstone Group LP,20.0,2282.140728,1067.070728
CTRA,Coterra Enercoterra energy Inc,85.0,1613.434074,-272.975926
FDS,FactSet Research Systems Inc,10.0,2198.261723,36.441723
GILD,Gilead Sciences Inc (All Sessions),10.0,891.158797,1458.728797
GOLD,Barrick Gold Corp (US),120.0,2895.403291,1029.183291


In [16]:
# find all the keys that map to the same ticker
ticker_valus = []
for k, v in symbols.items():
    ticker_valus.append(v)
# find duplicates in ticker_valus
for ticker in set(ticker_valus):
    keys = [k for k, v in symbols.items() if v == ticker]
    if len(keys) > 1:
        print(f"Ticker '{ticker}' is mapped from multiple company names: {keys}")

Ticker 'MP' is mapped from multiple company names: ['MP Materials Corp', 'MP Materials Corp (All Sessions)']
Ticker 'ME' is mapped from multiple company names: ['VG Acquisition Corporation (Ord)', '23ANDME Holding Co']
Ticker 'RTX' is mapped from multiple company names: ['Raytheon Technologies Corp (All Sessions)', 'RTX Corporation', 'RTX Corporation (All Sessions)']
Ticker 'OKTA' is mapped from multiple company names: ['Okta Inc', 'Okta Inc (Extended Hours)']


In [17]:
company_name_to_ticker = symbols
selected_markets = 'Okta Inc (Extended Hours)'
selected_ticker = company_name_to_ticker[selected_markets]
df_ticker_trade_history = df_positions[(df_positions['Market'] == selected_markets) & (df_positions['Activity']=="TRADE")]

In [18]:
df_ticker_trade_history = df_positions[(df_positions['Ticker'] == selected_ticker) & (df_positions['Activity']=="TRADE")]
df_ticker_trade_history

,TextDate,Time,Activity,Market,Direction,Quantity,Price,Currency,Consideration,Commission,Charges,Cost/Proceeds,Conversion rate,Order type,Venue ID,Settled?,Settlement date,Order ID,Ticker,Date
0,19-11-2025,19:46:00,TRADE,Okta Inc (Extended Hours),BUY,10.0,8018.0,USD,-801.8,0.0,0.0,-618.76,0.771715,LIMIT,XLON,Y,20-11-2025,07CLE:2064083~8779,OKTA,2025-11-19
134,04-11-2022,14:00:26,TRADE,Okta Inc,BUY,10.0,4580.0,USD,-458.0,0.0,0.0,-408.45,0.891816,LIMIT,XOFF,Y,08-11-2022,07CLE:1551939~17292,OKTA,2022-11-04
213,02-06-2021,20:30:05,TRADE,Okta Inc,BUY,10.0,21480.0,USD,-2148.0,0.0,0.0,-1523.19,0.709118,LIMIT,XOFF,Y,04-06-2021,07CLE:1254771~30550,OKTA,2021-06-02
219,03-05-2021,19:56:25,TRADE,Okta Inc,BUY,5.0,25688.0,USD,-1284.4,0.0,0.0,-927.82,0.722379,LIMIT,XOFF,Y,05-05-2021,07CLE:1239017~20968,OKTA,2021-05-03
227,25-02-2021,17:50:09,TRADE,Okta Inc,BUY,5.0,25800.0,USD,-1290.0,0.0,0.0,-921.06,0.714000,LIMIT,XOFF,Y,01-03-2021,07CLE:1195005~35445,OKTA,2021-02-25


In [19]:
import rewrite_plot_portfolio_weights as ppw
fig = ppw.plot_portfolio_weights(df_current_positions, [0]*len(df_current_positions), GBP)
fig.show()

KeyError: 'current position'